# Plot the Amtrak Thruway routes

Do these make sense?

Think about how to grab local buses that also travel the same origin / destination.

In [1]:
import geopandas as gpd
import pandas as pd

import _utils
from _utils import GCS_FILE_PATH, SELECTED_DATE

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
trip_gdf = gpd.read_parquet(
    f"{GCS_FILE_PATH}amtrak_thruway_trips_{SELECTED_DATE}.parquet")

gdf = _utils.aggregate_to_route(trip_gdf)

In [4]:
gdf = gdf.assign(
    route_length = gdf.geometry.to_crs("EPSG:2229").length / 5280
)

In [5]:
# Use a cut-off and filter down CA transit routes of at least 50 mi long
gdf.route_length.describe()

count     71.000000
mean     124.519811
std       88.204529
min        0.877638
25%       59.734679
50%      100.773935
75%      198.372044
max      335.443701
Name: route_length, dtype: float64

In [3]:
print(gdf.route_type.value_counts())
print(gdf.route_id.value_counts())

3    69
2     2
Name: route_type, dtype: int64
37329    61
42895     4
11317     2
42917     2
42896     2
Name: route_id, dtype: int64


In [4]:
def make_map(gdf: gpd.GeoDataFrame, route: str): 
    
    subset = gdf[gdf.route_id==route].reset_index(drop=True)
    
    print(f"Route: {route}    Route Type: {subset.route_type.iloc[0]}")

    display(subset[["origin_stop_name", "destination_stop_name"]].drop_duplicates())
    
    m = subset.explore("trip_id_nunique", categorical = False)
    display(m)
    
    display(subset)


In [5]:
amtrak_routes = gdf[gdf.route_type=='3'].route_id.unique().tolist()

for r in amtrak_routes:
    make_map(gdf, r)

Route: 37329    Route Type: 3


,origin_stop_name,destination_stop_name
0,Carmel Amtrak Bus Stop,Salinas
1,Sacramento,Truckee Amtrak Station
3,Sacramento,Auburn Amtrak
4,Truckee Amtrak Station,Sacramento
6,Salinas,Carmel Amtrak Bus Stop
7,Roseville Amtrak,Sacramento
8,Sacramento,Stockton
9,Stockton,Chico Amtrak
10,Redding Amtrak Bus Stop,Stockton
11,Stockton,Redding Amtrak Bus Stop


,calitp_itp_id,route_id,route_type,origin_stop_name,destination_stop_name,geometry,trip_id_nunique
0,13,37329,3,Carmel Amtrak Bus Stop,Salinas,"LINESTRING (-121.90880 36.53903, -121.89252 36...",2
1,13,37329,3,Sacramento,Truckee Amtrak Station,"LINESTRING (-121.50070 38.58400, -121.28629 38...",2
2,13,37329,3,Sacramento,Truckee Amtrak Station,"LINESTRING (-121.50070 38.58400, -121.28629 38...",2
3,13,37329,3,Sacramento,Auburn Amtrak,"LINESTRING (-121.50070 38.58400, -121.28629 38...",3
4,13,37329,3,Truckee Amtrak Station,Sacramento,"LINESTRING (-120.18539 39.32762, -120.95307 39...",2
...,...,...,...,...,...,...,...
56,13,37329,3,Santa Rosa Amtrak Bus Stop,Martinez,"LINESTRING (-122.72763 38.45560, -122.70975 38...",2
57,13,37329,3,Martinez,Napa Amtrak Bus Stop,"LINESTRING (-122.13878 38.01892, -122.25924 38...",1
58,13,37329,3,Martinez,Santa Rosa Amtrak Bus Stop,"LINESTRING (-122.13878 38.01892, -122.25924 38...",2
59,13,37329,3,Martinez,Santa Rosa Amtrak Bus Stop,"LINESTRING (-122.13878 38.01892, -122.25924 38...",2


Route: 42917    Route Type: 3


,origin_stop_name,destination_stop_name
0,Sacramento,South Lake Tahoe
1,South Lake Tahoe,Sacramento


...

,calitp_itp_id,route_id,route_type,origin_stop_name,destination_stop_name,geometry,trip_id_nunique
0,13,42895,3,Visalia,Santa Maria Amtrak Bus Stop,"LINESTRING (-119.28842 36.33189, -119.65182 36...",1
1,13,42895,3,Santa Maria Amtrak Bus Stop,Visalia,"LINESTRING (-120.41645 34.95161, -120.62927 35...",1
2,13,42895,3,Los Angeles,San Jose,"LINESTRING (-118.23678 34.05618, -118.44898 34...",1
3,13,42895,3,San Jose,Los Angeles,"LINESTRING (-121.90239 37.32993, -121.65675 36...",1


Route: 42896    Route Type: 3


,origin_stop_name,destination_stop_name
0,Barstow Amtrak Bus Stop,UC Santa Barbara Amtrak Bus Stop
1,UC Santa Barbara Amtrak Bus Stop,Barstow Amtrak Bus Stop


/opt/conda/lib/python3.10/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars


,calitp_itp_id,route_id,route_type,origin_stop_name,destination_stop_name,geometry,trip_id_nunique
0,13,42896,3,Barstow Amtrak Bus Stop,UC Santa Barbara Amtrak Bus Stop,"LINESTRING (-116.99843 34.89128, -117.02544 34...",1
1,13,42896,3,UC Santa Barbara Amtrak Bus Stop,Barstow Amtrak Bus Stop,"LINESTRING (-119.84830 34.41495, -119.69279 34...",1


In [6]:
amtrak_rail_routes = gdf[gdf.route_type=='2'].route_id.unique().tolist()

for r in amtrak_rail_routes:
    make_map(gdf, r)

Route: 11317    Route Type: 2


,origin_stop_name,destination_stop_name
0,Stockton,San Jose
1,San Jose,Stockton


/opt/conda/lib/python3.10/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars


,calitp_itp_id,route_id,route_type,origin_stop_name,destination_stop_name,geometry,trip_id_nunique
0,13,11317,2,Stockton,San Jose,"LINESTRING (-121.27900 37.95704, -121.26366 37...",4
1,13,11317,2,San Jose,Stockton,"LINESTRING (-121.90239 37.32993, -121.93658 37...",4


In [ ]:
# https://www.rome2rio.com/map/San-Jose/Stockton#r/Train/s/0


* MST runs longer routes
* Eastern Sierra Transit (Lancaster to Reno)
* Sonoma County Transit...or is it Golden Gate Transit 
* SolTrans

Also maybe grab a full week to get all the unique OD pairs

1. Follow Amtrak thruway (or same OD)
2. Don't follow Amtrak thruway but go longer distance
Longer route (> certain distance) get at Eastern Sierra
3. Long routes that travel on SHN on extended period of time


Use `pd.get_dummies` to find OD pairs

Also, wrangle df to be long, and then you can do the dissolve for OD buffers.


Draw buffer around rail lines, see which bus routes have significant overlay 